- SEARCH THE GEOJSON FILE TO GET UUID OF OBJECT THAT HAS COORDS BTWN MAX AND MIN X-Y

In [ ]:
import numpy as np
import json
# Import the Tree_Labels GeoJSON file
the_file = open("Tree_Labels.geojson")
tree_labs = json.load(the_file)

# Iterate through tree_labs to find the uuid
if "features" in tree_labs:
    for i in tree_labs["features"]:
        uuid_num = i["properties"]["uuid"]



In [ ]:
# Print the coordinates of each uuid that is under 10
for i in tree_labs["features"]:
    uuid_num = i["properties"]["uuid"]
    if uuid_num < 10:
        coords = i["geometry"]["coordinates"]
        print(coords)


In [ ]:
for i in tree_labs["features"]:
    uuid_num = i["properties"]["uuid"]
    if uuid_num == 1:
        coords_array = np.array(i["geometry"]["coordinates"])
        first_c = coords_array[0][0][0]
        if first_c[0] >= 279412.91098934:
            print("The first x-coordinate is: ")
            print(first_c[0])
            print(coords_array[0][0][0][0]) # This takes it down to the x coord of the first bounding box's first corner
        print("The first y-coordinate is: ")
        print(first_c[1])


In [ ]:
# Check the first x-y coordinate of each uuid to see if it is within bounds
total_poss = 0 # Number of possible bounding boxes in image
poss_uuim_list = [] # Empty list to put in uuim nums of possible bounding boxes in image
for i in tree_labs["features"]: # Check each bounding box
    uuid_num = i["properties"]["uuid"] # Stores the uuim of the current bounding box object we are in
    coords_array2 = np.array(i["geometry"]["coordinates"]) # Holds all the arrays of each vertex in the current bounding box
    if len(coords_array2)>0: # Make sure the coordinates arrays are not empty
        current_bbox = coords_array2[0][0][0] # First vertex of the current bounding box
        # TODO: Put image max and min in vars? - SEE BOXES BELOW
        # Make sure the x and y coordinates of the first vertex are within image bounds
        if 279255.7771 <= current_bbox[0] <= 279521.4213 and 693507.1507621396 <= current_bbox[
                1] <= 693690.5070378605:
            poss_uuim_list.append(uuid_num) # Appends uuim to list if first coordinates are within bounds
            total_poss+=1 # Plus 1 the number of possible bounding boxes in image



In [ ]:
# Go through geojson and get coordinates for each uuim on poss_uuim_list
full_coords_list = []
for i in poss_uuim_list:
    for j in tree_labs["features"]:
        if i == j["properties"]["uuid"]:
            full_coords_list.append(j["geometry"]["coordinates"])


In [ ]:
# Take certain coords from full_coords_list and put them into a seperate list with following pattern:
#  image_path (name of image); xmin; ymin; xmax; ymax; label (e.g. 'tree')
for_csv = [] # The list of lists for bounding boxes in image
image_name = "theImageName"
label = "tree"

for i in full_coords_list:
    # Get mins and maxs of bounding box
    geo_xmin = i[0][0][0][0]
    geo_ymin = i[0][0][0][1]
    geo_xmax = i[0][0][2][0]
    geo_ymax = i[0][0][1][1]

    # Put everything in order for line in csv into a list
    temp_list = [image_name, geo_xmin, geo_ymin, geo_xmax, geo_ymax, label]
    # Add that bounding box list to the list of all bounding boxes in image
    for_csv.append(temp_list)


In [ ]:
# Calculate pixel per geocoord metre in the image
import rasterio as rio
the_image = rio.open('shouldbeworking2.tif')

# Width and Height of image in pixels
img_width_px = the_image.width
img_height_px = the_image.height

# Geolocation boundaries of image
img_left_bounds = the_image.bounds.left
img_right_bounds = the_image.bounds.right
img_top_bounds = the_image.bounds.top
img_bottom_bounds = the_image.bounds.bottom

# Width and Height of image in geo-coord metres
x_coords_span = img_right_bounds - img_left_bounds
y_coords_span = img_top_bounds - img_bottom_bounds

# Image pixels per geo-coord metre
# todo: Limit result to certain s.f.?
px_per_m = img_width_px / x_coords_span

# for i in for_csv:
#     print(i)


In [ ]:
# Calculate image pixel min-max for bounding box and replace their geocoord equivalent in for_csv List

for i in range(len(for_csv)):
    # The xmin in pixels: geocoords of left edge of bounding box minus geocoords of left edge of image
    # multiplied by pixels per metre to turn geocoord difference into pixel difference

    px_xmin = (for_csv[i][1] - img_left_bounds) * px_per_m
    px_ymin = (img_top_bounds - for_csv[i][4]) * px_per_m # Remember image pixel coords start top-left not bottom-right
    px_xmax = (img_right_bounds - for_csv[i][3]) * px_per_m
    px_ymax = (img_top_bounds - for_csv[i][2]) * px_per_m # Remember image pixel coords start top-left not bottom-right

    # Now replace items in positions 1 through 4 in the List
    for_csv[i][1:5] = [px_xmin, px_ymin, px_xmax, px_ymax]



In [ ]:
# TODO: Change coords from geo to image - See W3Schools, change list items, Change Item Value and Change a Range of Item Values
# todo: pixels per coord metre = pixels divided by difference of coord max and min
# TODO: THIS IS FOR THE FIRST BOUNDING BOX
# x geocoord = 265.6442 m (?) - SPANNING THE X AXIS
# y geocoord = 183.3562757209 m (?) - SPANNING THE Y AXIS
# x pixels = 2150 - IMAGE WIDTH IN PIXELS
# y pixels = 1484 - IMAGE HEIGHT IN PIXELS
# 8.09353262747 pixels per metre - x axis
# 8.09353262747 pixels per metre - y axis
# xmin is 368 px
# ymin is 984 px - REMEMBER IMAGE COORDS GO FROM TOP LEFT NOT BOTTOM LEFT!!
# xmax is 1709 px
# ymax is 1083 px - REMEMBER IMAGE COORDS GO FROM TOP LEFT NOT BOTTOM LEFT!!

In [ ]:
# TODO: Put for_csv into json format
import pandas

columns = ["image_path", "xmin", "ymin", "xmax", "ymax", "label"]
# TODO: Should I just delete the first column, the one with the index from 0?
#index = [i[0] for i in for_csv] # Makes the first element the index, in this case image_path will be index
not_index_list = [i for i in for_csv]
pd = pandas.DataFrame(not_index_list, columns=columns)

# Create csv with column headers and index
pd.to_csv("bounding.csv")
# print(pd)

